In [44]:
import numpy as np
import pandas as pd
import datascience
from datascience.predicates import are
import shapely
from shapely.geometry import Polygon
from shapely.geometry import LineString

In [45]:
x = [ -71.20197, -70.96679]
y = [42.291441, 42.420578]
x_cell = 50
y_cell = 50


x_min = -71.20197 #x corresponds to longitude
x_max = -70.96679
y_min = 42.291441 #y corresponds to latitude
y_max = 42.420578

#Create ticks
x_s = np.linspace(x_min, x_max, x_cell + 1)
y_s = np.linspace(y_min, y_max, y_cell + 1)

#Create grid
x_coord, y_coord = np.meshgrid(x_s, y_s)

In [46]:
exp = datascience.Table.read_table("highwaydata/exp_high_sub.csv")
art = datascience.Table.read_table("highwaydata/art_high_sub.csv")

In [47]:
'''Function Description: given the x and y coordinates of a grid, turns each grid element into a polygon. I will move from left to right, bottom to top

Input: x-coords, y-coords is a list of lists of x and y coordinates for the grid

Output: a list of polygons, one for each grid '''

def make_polygon(x_coords, y_coords):
    polygons = []
    for a in range(0, len(x_coords[0])-1, 1):
        x = x_coord[0][a:a+2]
        #grabs each x-coordinate pair for each grid element
        for b in range(0, len(y_coords)-1, 1):
            grid = []
            #initiate list to store points for each grid element
            y = tuple([y[0] for y in y_coords[b:b+2]])
            #for each x-coordinate pair, get all corresponding y-coordinates
            for j in range(0, len(y)):
                grid.append((x[0], y[j]))
                grid.append((x[1], y[j]))
                #append grid points of polygon into the grid list 
                g= [grid[0]] + grid[2:4] + [grid[1]]
                #reorder for shapely's Polygon function to read
            polygon = Polygon(g)
            polygons.append(polygon)
        #turn grid into shapely polygon and append to polygon list
    return(polygons)

polygons = make_polygon(x_coord, y_coord)

In [48]:
#processing for express highways
data_beg = []
data_end = []

begin_lat = exp["begin lat"]
begin_long = exp["begin long"]
end_lat = exp["end lat"]
end_long = exp["end long"]


for x in range(0,len(begin_lat)) :
    data_beg.append(tuple([begin_long[x], begin_lat[x]]))
    data_end.append(tuple([end_long[x], end_lat[x]]))

In [49]:
#processing for arterial highways
data_beg_a = []
data_end_a = []

begin_lat_a = art["begin lat"]
begin_long_a = art["begin long"]
end_lat_a = art["end lat"]
end_long_a = art["end long"]

for x in range(0,len(begin_lat_a)) :
    data_beg_a.append(tuple([begin_long_a[x], begin_lat_a[x]]))
    data_end_a.append(tuple([end_long_a[x], end_lat_a[x]]))

In [50]:

'''Function Description: given the start and end points of a line and a polygon, says whether the line goes through that grid

Input: start_loc and end_loc are tuples of latitude, longitude. grid is a list of 4 latitude, longitude tuples detailing a rectangular grid

Output: Boolean'''

def make_lines(begin_lst, end_lst):
        return ([LineString([begin_lst[i], end_lst[i]]) for i in np.arange(0, len(begin_lst), 1)])

In [51]:
'''Function Description: given the start and end points of a line and a full grid, tells how many grid elements the line go through 

Input: line is a linestring shapely object. grid is a list of latitude, longitude tuples detailing a rectangular grid

Output: Boolean'''
#given the x-coords and y-coords of our grid, tells how many grid it goes through

def how_many(line, poly_list):
    grids_thru = 0
    for poly in poly_list:
        thru = poly.intersects(line)
        inside = poly.contains(line)
        if thru or inside:
            grids_thru += 1
    return (grids_thru)

In [52]:
lines = make_lines(data_beg, data_end)

grids_thru = []
for i in lines:
    grids_thru.append(how_many(i, polygons))
    
#in final data frame: data_beg, data_end, thru_grids, the four grid points
d = {"begin": data_beg, "end": data_end, "grids through": grids_thru}
final_exp = pd.DataFrame(data=d)

In [53]:
lines_a = make_lines(data_beg_a, data_end_a)

grids_thru_a = []
for i in lines_a:
    grids_thru_a.append(how_many(i, polygons))
    
d_a = {"begin": data_beg_a, "end": data_end_a, "grids through": grids_thru_a}
final_art = pd.DataFrame(data=d_a)

In [40]:
#Scratch code

grid = []
x = x_coord[0][0:2]
y = tuple([y[0] for y in y_coord[0:2]])
for j in range(0, len(y)):
    grid.append((x[0], y[j]))
    grid.append((x[1], y[j]))
g= [grid[0]] + grid[2:4] + [grid[1]] 
polygon = Polygon(g)
a = (-71.20197, 41)#(exp["begin lat"][0],  exp["begin long"][0])
b = (-71.20197, 43)#(exp["end lat"][0],  exp["end long"][0])
line = LineString([a, b])
polygon.intersects(line)

#Generalizing a url string to be read, adjusting for varying latitudes and longitudes
#Plan of attack: take the center of the grid and check at that point
#Vertices of the grid:

def points_in_box(min_lon, max_lon, min_lat, max_lat):
    #Take the center point of this grid box
    lon = float((min_lon + max_lon)/2)
    lat = float((min_lat + max_lat)/2)

    #read page
    begin_lat = exp["begin lat"]
    begin_long = exp["begin long"]
    end_lat = exp["end lat"]
    end_long = exp["end long"]
    #This list is going to hold tuples that will be the ordered pairs of all the nearby stops
    data_beg, data_end = [], []
    for x in range(0,len(begin_lat)) :
        data_beg.append(tuple([begin_lat[x], begin_long[x]]))
        data_end.append(tuple([end_lat[x], end_long[x]]))

    #We now have a list of ordered pairs that describe the locations of all the nearby stops
    #Next we have to determine if there are stops contained within the parameters of a given grid box
    #And, if so, count how many
    startsingrid = 0
    endsingrid = 0
    for y in range(0, len(data_beg)):
        if (data_beg[y][0]>= min_lat and data_beg[y][0]<=max_lat) and (data_beg[y][1]>= min_lon and data_beg[y][1]<=max_lon):
            startsingrid = 1 + startsingrid
        if (data_end[y][0]>= min_lat and data_end[y][0]<=max_lat) and (data_end[y][1]>= min_lon and data_end[y][1]<=max_lon):
            endsingrid = 1 + endsingrid

    #Only part that will need to be changed depending on our needs: What is printed/ returned at the end
    #print("Center of Grid: (" + str(lat) + ", " + str(lon) + ")" )
    print("Start Locations in this gridbox: "+ str(startsingrid)+ ", End Locations in this gridbox: "+ str(endsingrid))
#end of method